<center>

# **Box Stacking Problem**

---

### **Descripcion del Problema:**

Se tiene un **conjunto** de **cajas B** con **dimensiones** **largo**, **ancho** y **altura**. 

El **objetivo** es **formar** la **torre más alta** que se **puede generar** con ellas, con la **única condición** de que **no** se puede **colocar** una **caja con área mayor arriba** de una **caja con área menor**. 

Encontrar un **algoritmo** con el cual **logren hacerlo** de la **forma más eficiente posible**.

---

### **El Problema del Apilamiento de Cajas**

El **Problema de Apilamiento de Cajas** (**Box Stacking Problem**) es un **problema** de **optimización combinatoria** que se utiliza para **encontrar** la **altura máxima** que se puede lograr al **apilar cajas rectangulares en una pila**.

Suponiendo que se tienen **n cajas**, cada una con una **altura h**, un **ancho w** y una **profundidad d**. El **objetivo** es **apilar** las **cajas** de tal manera que se **maximice** la **altura total de la pila**, teniendo en cuenta que las **cajas sólo se pueden apilar si su base es menor en altura**, **ancho** y **profundidad** que la **caja** que está **debajo**.

El **problema del apilamiento de cajas** tiene **aplicaciones** prácticas en la **logística**, en la **optimización de espacios de almacenamiento**, y en la **planificación** de la **distribución de productos en estantes** y **paletas** en el **transporte** y **almacenamiento de mercancías**.

---

### **Algoritmo Voraz**

+ Los **algoritmos voraces (greedy algorithms)**, son algoritmos que **resuelven problemas** mediante la **toma de decisiones locales óptimas** en cada paso, **sin considerar** las posibles **consecuencias futuras** de esas decisiones. 

+ En otras palabras, **en cada paso** del algoritmo, **se toma la mejor decisión posible** basada en la información disponible en ese momento, con la esperanza de que **esa decisión conduzca** a una **solución óptima global**.

#### **Algoritmo Voraz en Lenguaje Natural** 

1. **Ordenar** las **cajas** en **base** a una **dimensión**, por ejemplo, **altura**.
2. **Iniciar** con la **caja más alta** en la **parte inferior** y **colocar** la **siguiente caja más alta** que **pueda** ser **apilada sobre ella**.
3. **Continuar** **colocando** las **cajas más altas posibles** en la **parte superior** de la **pila**, **siempre y cuando** las **dimensiones** de la **caja superior** sean **menores que** las **dimensiones** de la **caja inferior**.
4. **Repetir** los **pasos** **2** y **3** **hasta** que se hayan **utilizado** **todas** las **cajas**.


In [3]:
#Importamos el módulo numpy para manejar de manera fácil los elementos de las listas.
import numpy as np

def crear_cajas(num_cajas, tamaño_limite):
    # Crear cajas de manera aleatoria con numpy
    cajas = np.random.randint(1, tamaño_limite+1, size=(num_cajas, 3))
    return cajas

def apilamiento_cajas(cajas):
    # Ordenar las cajas por altura (tercera dimensión)
    cajas_ordenadas = cajas[np.argsort(cajas[:, 2])[::-1]]
    
    n = len(cajas_ordenadas)
    alturas = cajas_ordenadas[:, 2]
    bases = cajas_ordenadas[:, :2]
    dp = np.zeros(n)
    
    # Calcular la altura máxima que se puede alcanzar al agregar la i-ésima caja en la pila
    for i in range(n):
        for j in range(i):
            if np.all(bases[i] < bases[j]):
                dp[i] = max(dp[i], dp[j])
        dp[i] += alturas[i]
        
    # Encontrar la altura máxima posible
    altura_maxima = np.max(dp)
    
    # Construir la pila de cajas
    pila_cajas = []
    altura_actual = altura_maxima
    for i in range(n - 1, -1, -1):
        if dp[i] == altura_actual:
            pila_cajas.append(cajas_ordenadas[i])
            altura_actual -= cajas_ordenadas[i, 2]
    
    # Devolver la altura máxima
    return altura_maxima

print(" Cajas ".center(125, "-"))
cajas = crear_cajas(1000, 15)
print(cajas)
print(" Resultado ".center(125, "-"))
print(f"Altura máxima: {apilamiento_cajas(cajas)}")


----------------------------------------------------------- Cajas -----------------------------------------------------------
[[15  5  3]
 [ 2  2 15]
 [13  7  1]
 ...
 [ 4 15  9]
 [ 6  4  3]
 [ 8  8  5]]
--------------------------------------------------------- Resultado ---------------------------------------------------------
Altura máxima: 128.0


---

### **Programación Dinámica**

+ La **programación dinámica** es una **técnica** utilizada en informática y matemáticas para **resolver problemas complejos dividiéndolos** en otros **más pequeños** y **resolviendo** cada problema, **guardando** sus **resultados** y **reutilizándolos** en lugar de leerlos nuevamente.  

+ La **programación dinámica** ofrece **beneficios** como la **optimización de tiempo**, la **garantía de solución óptima**, la **reutilización de resultados**, la **versatilidad** y la **escalabilidad**. Estos beneficios hacen que sea una **técnica** valiosa para **resolver problemas computacionales** de manera **eficiente** y **efectiva**.

#### **Algoritmo de Programación Dinámica en Lenguaje Natural**

1. **Generar** un **conjunto** de **todas** las posibles **rotaciones** de las **cajas**.
2. **Ordenar** las **cajas** en **orden no creciente** de **área** de **base**.
3. **Crear** una **matriz tridimensional** donde la entrada **matriz[i][j][k] representa** la **altura máxima** que se puede **obtener** al **colocar** la **caja i** en la **parte superior** de la **caja j** y en el **lugar k** para **todas** las **combinaciones (i,j,k)** de **cajas**.
4. **Inicializar** todas las entradas de la **matriz con ceros**.
5. **Recorrer** la **matriz tridimensional** y se **calcula** el **valor** de **cada entrada matriz[i][j][k]** de la siguiente manera:
    1. **matriz[i][j][k] = matriz[j][k][máxima_altura] + altura[i]** donde "**máxima_altura**" es el **valor** de **k** que **maximiza** la **altura total** y **cumple** las **restricciones** de **tamaño** de la **base de la caja**.
6. **Encuentra** la **altura máxima** de la **pila** como el **máximo valor** de la **matriz**.

In [4]:
#Importamos el módulo numpy para manejar de manera fácil los elementos de las listas.
import numpy as np

def calcular_max_altura(cajas):
    # Obtener dimensiones de cada caja
    n = len(cajas)
    dimensiones = np.zeros((n*3, 3))
    for i in range(n):
        d1, d2, d3 = cajas[i]
        dimensiones[i*3] = [d1, d2, d3]
        dimensiones[i*3+1] = [d2, d1, d3]
        dimensiones[i*3+2] = [d3, d1, d2]
    n *= 3

    # Ordenar cajas por área de base en orden descendente
    indices = np.argsort(dimensiones[:, 0] * dimensiones[:, 1])[::-1]
    dimensiones = dimensiones[indices]

    # Calcular altura máxima usando programación dinámica
    dp = np.zeros(n)
    dp[0] = dimensiones[0][2]
    for i in range(1, n):
        max_altura = 0
        for j in range(i):
            if (dimensiones[i][0] < dimensiones[j][0] and dimensiones[i][1] < dimensiones[j][1]):
                max_altura = max(max_altura, dp[j])
        dp[i] = max_altura + dimensiones[i][2]

    return np.max(dp)

# Ejemplo de uso
print(" Cajas ".center(125, "-"))
print(cajas)
print(" Resultado ".center(125, "-"))
print(f"Altura máxima: {calcular_max_altura(cajas)}")

----------------------------------------------------------- Cajas -----------------------------------------------------------
[[15  5  3]
 [ 2  2 15]
 [13  7  1]
 ...
 [ 4 15  9]
 [ 6  4  3]
 [ 8  8  5]]
--------------------------------------------------------- Resultado ---------------------------------------------------------
Altura máxima: 209.0


---